<h1>Описание проекта</h1>

Нужно защитить данные клиентов страховой компании. Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Загрузка данных

In [1]:
#импорт библиотек
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('datasets/insurance.csv')

In [3]:
# Отображение верхней части df
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
#типы данных столбцов и непустые значения
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
#проверка дубликатов
data[data.duplicated()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


**Вывод:** В данных нет пропущенных значений, в стобцах `Возраст` и `Зарплата` можно поменять тип данных на целые числа при необходимости. Были найдены `153 дубликата`, но нельзя определить задвоение это или просто похожие люди. Для точного определения не хвадает идентификатора(user_id).

Даже с этими недочётами данные нас устраивают, потому что подбирать наилучшую модель не требуется.

## Умножение матриц

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Обратимые матрицы** - матрицы, для которых можно найти обратные матрицы.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $E$ - единичная матрица

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** 
b. Не изменится.

**Обоснование:** 
$$
a =X(X^T X)^{-1} X^T y
$$

$$
a1 = XP((XP)^T XP)^{-1}(XP)^Ty
$$

$$
a1 = XP(P)^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

Обратная для квадратной матрицы P  — матрица P с верхним индексом -1, произведение которой на P равно единичной матрице.

$$
PP^{-1} = P^{-1}P = E
$$

$$
a1 = XE(X^TX)^{-1}EX^Ty
$$

Если любую матрицу X умножить на единичную (или наоборот), получится эта же матрица X

$$
XE = EX = X
$$

$$
a1 = X(X^T X)^{-1} X^T y
$$

$$
a = a1
$$

## Алгоритм преобразования

In [6]:
#отделяем фичи и таргет
x = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

**Алгоритм преобразования:**

1. Создаём произвольную обратимую матрицу подходящего нам размера

In [7]:
#создаём случайную обратимую матрицу
P = np.random.randint(-100000, 666, (4, 4))
print(P)

[[-74485 -66875 -35981 -43024]
 [-45760 -20191 -31060 -59525]
 [-60565 -11912 -14997 -65309]
 [-80207 -69836 -25613  -6091]]


2. Находим скалярное произведение полученной матрицы в п.1 и наших признаков, которые необходимо преобразовать

In [8]:
#шифрование фичи x
x_trans = np.array(x @ P)
print(x_trans)

[[-3.00605485e+09 -5.91799742e+08 -7.45186254e+08 -3.24181604e+09]
 [-2.30365517e+09 -4.53654622e+08 -5.71340373e+08 -2.48448624e+09]
 [-1.27319204e+09 -2.50737539e+08 -3.15837740e+08 -1.37321522e+09]
 ...
 [-2.05422911e+09 -4.04360292e+08 -5.09070726e+08 -2.21517778e+09]
 [-1.98179733e+09 -3.90242985e+08 -4.91198040e+08 -2.13697515e+09]
 [-2.46037497e+09 -4.84329259e+08 -6.09809474e+08 -2.65326122e+09]]


Алгоритм преобразования готов! Данные надёжно спрятаны, осталось проверить качество метрики.

## Проверка алгоритма

In [9]:
# модель линейной регрессии
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()

In [10]:
model.fit(x, y)
predictions = model.predict(x)
print('Метрика R2 до преобразования:', r2_score(y, predictions))

Метрика R2 до преобразования: 0.42494550286668


In [11]:
model.fit(x_trans, y)
predictions = model.predict(x_trans)
print('Метрика R2 после преобразования:', r2_score(y, predictions))

Метрика R2 после преобразования: 0.42494550286004584


## Общий вывод

Мы смогли защитить данные клиентов страховой компании.\ 
Разработав метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.\
При преобразовании качество моделей машинного обучения не ухудшилось.